# Аналіз якості даних Life Expectancy (WHO)

Цей ноутбук демонструє перевірку якості даних: пропущені значення, дублікати, викиди.

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_load import load_data
from data_quality_analysis import (
    check_missing_values,
    check_duplicates,
    detect_outliers_iqr,
    generate_quality_report,
    print_quality_report
)

plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Завантаження даних

In [ ]:
df = load_data()
print(f"Розмір датасету: {df.shape}")
df.head()

## 2. Повний звіт про якість даних

In [ ]:
report = generate_quality_report(df)
print_quality_report(report)

## 3. Детальний аналіз пропущених значень

In [ ]:
missing_df = check_missing_values(df)

if len(missing_df) > 0:
    plt.figure(figsize=(12, 6))
    plt.barh(missing_df['column'], missing_df['missing_percentage'])
    plt.xlabel('Відсоток пропущених значень (%)')
    plt.title('Пропущені значення по стовпцях')
    plt.tight_layout()
    plt.show()
    
    display(missing_df)
else:
    print("✓ Пропущених значень не знайдено")

## 4. Перевірка дублікатів

In [ ]:
dup_info = check_duplicates(df)

print(f"Всього дублікатів: {dup_info['total_duplicates']}")
print(f"Відсоток дублікатів: {dup_info['duplicate_percentage']}%")

if dup_info['total_duplicates'] > 0:
    print("\nПриклади дублікатів:")
    display(dup_info['duplicate_rows'].head(10))

## 5. Виявлення викидів

In [ ]:
# Аналіз викидів для Life expectancy
column = 'Life expectancy '

if column in df.columns:
    outliers, stats = detect_outliers_iqr(df.dropna(subset=[column]), column)
    
    print(f"Аналіз викидів для '{column}':")
    print(f"  Q1: {stats['Q1']:.2f}")
    print(f"  Q3: {stats['Q3']:.2f}")
    print(f"  IQR: {stats['IQR']:.2f}")
    print(f"  Lower bound: {stats['lower_bound']:.2f}")
    print(f"  Upper bound: {stats['upper_bound']:.2f}")
    print(f"  Викидів: {stats['outliers_count']} ({stats['outliers_percentage']}%)")
    
    # Візуалізація
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Boxplot
    axes[0].boxplot(df[column].dropna())
    axes[0].set_ylabel(column)
    axes[0].set_title('Box Plot')
    
    # Histogram
    axes[1].hist(df[column].dropna(), bins=30, edgecolor='black')
    axes[1].axvline(stats['lower_bound'], color='r', linestyle='--', label='Lower bound')
    axes[1].axvline(stats['upper_bound'], color='r', linestyle='--', label='Upper bound')
    axes[1].set_xlabel(column)
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Histogram')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 6. Кореляційний аналіз

In [ ]:
# Матриця кореляції для числових змінних
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Матриця кореляції')
plt.tight_layout()
plt.show()

# Топ корельованих з Life expectancy
if 'Life expectancy ' in correlation_matrix.columns:
    life_exp_corr = correlation_matrix['Life expectancy '].sort_values(ascending=False)
    print("\nТоп-10 факторів, що корелюють з Life expectancy:")
    print(life_exp_corr.head(10))

## 7. Висновки

На основі аналізу якості даних можна зробити висновки про:
- Загальний стан датасету
- Наявність та кількість пропущених значень
- Потенційні проблеми з даними
- Необхідність попередньої обробки